In [1]:
# Cell 1: Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report, roc_auc_score, precision_score, recall_score, roc_curve, precision_recall_curve, matthews_corrcoef
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LeakyReLU, Dropout, LayerNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

print("Libraries loaded successfully.")

2024-09-19 15:38:52.422902: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-19 15:38:52.442030: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-19 15:38:52.463058: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-19 15:38:52.469353: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-19 15:38:52.485656: I tensorflow/core/platform/cpu_feature_guar

Libraries loaded successfully.


In [2]:
# Cell 2: Load and preprocess dataset
df = pd.read_csv('merged_data_clean.csv')
df = df[(df['Source'] == 'ToxiM') | (df['Source'] == 'MolToxPred')]
categorical_columns = ['SMILES', 'Source']
df = df.drop(columns=categorical_columns)
Y = df['Toxicity']
X_pca_df = pd.read_csv('X_pca_clean_with_boruta_ToxiM_and_MolToxPred.csv')
# X_pca_df = pd.read_csv('X_pca_clean_with_boruta_ToxiM_and_MolToxPred.csv')
# Handle class imbalance using SMOTE
# Split the original data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X_pca_df, Y, test_size=0.2, random_state=42, shuffle=True, stratify=Y)

# Apply SMOTE only on the training data
smote = SMOTE(random_state=42)
X_train_resampled, Y_train_resampled = smote.fit_resample(X_train, Y_train)

print("Data preprocessing completed.")

Data preprocessing completed.


In [3]:
# Cell 3: Define and train Deep Neural Network with Keras Tuner
from kerastuner.tuners import BayesianOptimization

def build_model(hp):
    model = Sequential()
    for i in range(hp.Int('num_layers', 2, 6)):
        model.add(Dense(units=hp.Int(f'units_{i}', min_value=32, max_value=512, step=32), 
                        kernel_regularizer=tf.keras.regularizers.l2(0.01)))
        model.add(LeakyReLU(alpha=0.01))
        model.add(LayerNormalization())
        model.add(Dropout(hp.Float(f'dropout_{i}', min_value=0.3, max_value=0.5, step=0.05)))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer=Adam(learning_rate=hp.Float('learning_rate', 1e-4, 1e-2, sampling='LOG')), 
                  loss='binary_crossentropy', 
                  metrics=['accuracy'])
    return model

tuner = BayesianOptimization(build_model, 
                             objective='val_accuracy', 
                             max_trials=20, 
                             executions_per_trial=2, 
                             directory='tuner_dir', 
                             project_name='hyperparameter_tuning_bayes')

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10, min_lr=0.0001)
early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

tuner.search(X_train, Y_train, epochs=50, validation_split=0.2, callbacks=[reduce_lr, early_stopping])

best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
dnn_model = tuner.hypermodel.build(best_hps)
history = dnn_model.fit(X_train, Y_train, epochs=300, batch_size=32, validation_split=0.2, verbose=1, callbacks=[reduce_lr, early_stopping])

print("Deep Neural Network trained.")

Reloading Tuner from tuner_dir/hyperparameter_tuning_bayes/tuner0.json


/tmp/ipykernel_39665/764455109.py:2: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import BayesianOptimization
2024-09-19 15:40:34.079720: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
/home/angadjeet22071/miniconda3/lib/python3.11/site-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Epoch 1/300
282/282 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.6495 - loss: 5.6477 - val_accuracy: 0.7845 - val_loss: 4.4537 - learning_rate: 1.6803e-04
Epoch 2/300
282/282 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.7750 - loss: 4.2110 - val_accuracy: 0.8072 - val_loss: 3.4740 - learning_rate: 1.6803e-04
Epoch 3/300
282/282 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.8028 - loss: 3.2871 - val_accuracy: 0.8196 - val_loss: 2.7730 - learning_rate: 1.6803e-04
Epoch 4/300
282/282 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.8306 - loss: 2.5906 - val_accuracy: 0.8183 - val_loss: 2.2739 - learning_rate: 1.6803e-04
Epoch 5/300
282/282 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.8379 - loss: 2.1325 - val_accuracy: 0.8223 - val_loss: 1.9347 - learning_rate: 1.6803e-04
Epoch 6/300
282/282 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.8469 - loss: 1.7876 - val_accuracy: 0.8254 - val_loss: 1.6566 - learning_rate: 1.6803e-04
Epoch 7/300
282/282 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - acc

In [4]:
models = {
    'LR': LogisticRegression(random_state=42,max_iter=200, C =0.01),
    'DT': DecisionTreeClassifier(random_state = 42,max_depth=10),
    'RF': RandomForestClassifier(random_state = 42, n_estimators=100),
    'GB': GradientBoostingClassifier(random_state=42,learning_rate=0.2),
    'XGB': XGBClassifier(random_state = 42,use_label_encoder=False, eval_metric='logloss', learning_rate=0.1),
    'SVM': SVC(probability=True, C=1),
    'KNN': KNeighborsClassifier(n_neighbors=7),
    'NB': GaussianNB()
}

fitted_models = {}
for model_name, model in models.items():
    # if model_name not in ["SVM"]:
    #         continue
    print("This model is running", model_name)
    model.fit(X_train, Y_train)  # Fit the model to the training data
    fitted_models[model_name] = model  # Store the trained model
fitted_models['DNN'] = dnn_model;
print("Traditional machine learning models trained.")

This model is running LR
This model is running DT
This model is running RF
This model is running GB
This model is running XGB
This model is running SVM
This model is running KNN
This model is running NB
Traditional machine learning models trained.


In [5]:
import pickle

fitted_models = {}

# Iterate over models, only fitting the "SVM" model
for model_name, model in models.items():
    # if model_name not in ["SVM"]:
    #     continue
    print("This model is running:", model_name)
    model.fit(X_train, Y_train)  # Fit the model to the training data
    fitted_models[model_name] = model  # Store the trained model

# Add the DNN model separately
fitted_models['DNN'] = dnn_model

print("Traditional machine learning models trained.")

# Save the fitted models to a file
with open('fitted_models.pkl', 'wb') as file:
    pickle.dump(fitted_models, file)

print("Fitted models saved to 'fitted_models.pkl'.")


This model is running: LR
This model is running: DT
This model is running: RF
This model is running: GB
This model is running: XGB
This model is running: SVM
This model is running: KNN
This model is running: NB
Traditional machine learning models trained.
Fitted models saved to 'fitted_models.pkl'.


In [6]:
from sklearn.svm import SVC, LinearSVC

# Assuming model is an instance of SVM
if isinstance(model, SVC):
    print(f"Kernel used: {model.kernel}")
    if model.kernel == 'linear':
        print("The SVM model is linear.")
    else:
        print("The SVM model is non-linear.")
elif isinstance(model, LinearSVC):
    print("The SVM model is linear.")
else:
    print("Unknown model type.")


Unknown model type.


In [7]:
# Cell 5: Evaluate all models and store metrics
def evaluate_models(models, X_train, X_test, Y_train, Y_test):
    results = []
    for model_name, model in models.items():
        print(f"Evaluating {model_name}...")
        if model_name == 'DNN':
            train_pred = (model.predict(X_train) > 0.5).astype(int)
            test_pred = (model.predict(X_test) > 0.5).astype(int)
            train_pred_prob = model.predict(X_train).ravel()
            test_pred_prob = model.predict(X_test).ravel()
        else:
            train_pred = model.predict(X_train)
            test_pred = model.predict(X_test)
            train_pred_prob = model.predict_proba(X_train)[:, 1]
            test_pred_prob = model.predict_proba(X_test)[:, 1]
        
        metrics = {
            'Model': model_name,
            'Train Accuracy': accuracy_score(Y_train, train_pred),
            'Test Accuracy': accuracy_score(Y_test, test_pred),
            'Train F1': f1_score(Y_train, train_pred),
            'Test F1': f1_score(Y_test, test_pred),
            'Train MCC': matthews_corrcoef(Y_train, train_pred),
            'Test MCC': matthews_corrcoef(Y_test, test_pred),
            'Train ROC AUC': roc_auc_score(Y_train, train_pred_prob),
            'Test ROC AUC': roc_auc_score(Y_test, test_pred_prob),
            'Train Precision': precision_score(Y_train, train_pred),
            'Test Precision': precision_score(Y_test, test_pred),
            'Train Recall': recall_score(Y_train, train_pred),
            'Test Recall': recall_score(Y_test, test_pred)
        }
        results.append(metrics)
    
    results_df = pd.DataFrame(results)
    results_df.to_csv("model_metrics_latest_good_smote.csv")
    print("Model metrics saved to 'model_metrics.csv'.")

evaluate_models({'DNN': dnn_model, **fitted_models}, X_train, X_test, Y_train, Y_test)

Evaluating DNN...
352/352 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
352/352 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Evaluating LR...
Evaluating DT...
Evaluating RF...
Evaluating GB...
Evaluating XGB...
Evaluating SVM...
Evaluating KNN...
Evaluating NB...
Model metrics saved to 'model_metrics.csv'.


In [8]:
# import numpy as np
# import lime
# from lime.lime_tabular import LimeTabularExplainer
# import pandas as pd

# print("Applying LIME analysis...")

# # Create a LIME explainer
# explainer = LimeTabularExplainer(
#     training_data=np.array(X_train),
#     feature_names=X_pca_df.columns,
#     class_names=['Non-Toxic', 'Toxic'],
#     mode='classification'
# )

# # Select a test instance to explain
# idx = 0  # Index of the instance you want to explain
# instance = X_test.iloc[idx]  # Use .iloc for integer-location based indexing

# # Define the predict function for each model to return probabilities
# def predict_fn(x, model):
#     preds = model.predict(x)
#     # Normalize the output if necessary
#     if len(preds.shape) == 1:  # Binary classification case for DNN
#         preds = np.hstack((1 - preds.reshape(-1, 1), preds.reshape(-1, 1)))
#     elif preds.shape[1] == 1:  # Binary classification case for other models
#         preds = np.hstack((1 - preds, preds))
#     return preds

# # Function to apply LIME and generate explanations
# def apply_lime_and_save(models, instance, explainer):
#     for model_name, model in models.items():
#         print(f"Applying LIME for {model_name}...")
#         exp = explainer.explain_instance(
#             data_row=instance,
#             predict_fn=lambda x: predict_fn(x, model),
#             num_features=10
#         )
#         exp.save_to_file(f'lime_explanation_{model_name}.html')
#         print(f"LIME explanation for {model_name} saved to 'lime_explanation_{model_name}.html'.")

# apply_lime_and_save({'DNN': dnn_model, **fitted_models}, instance, explainer)


In [9]:
# import shap
# import json
# import matplotlib.pyplot as plt

# # Iterate through each fitted model to perform SHAP analysis
# shap_values_dict = {}
# processed_models = []

# # Summarize the background data using k-means with fewer clusters
# background = shap.kmeans(X_test, 5)  # Reducing to 5 clusters for faster computation

# # Use a subset of X_test for SHAP analysis
# X_test_subset = X_test.sample(n=1, random_state=42)  

# for model_name, model in fitted_models.items():
#     try:
#         if model_name not in [ "SVM"]:
#             continue
#         print(f"Performing SHAP analysis for {model_name}...")

#         # Use KernelExplainer for SVM
#         if model_name == "SVM":
#             explainer = shap.KernelExplainer(model.predict, background)
#         else:
#             explainer = shap.Explainer(model.predict, X_test)

#         # Calculate SHAP values using the subset of X_test
#         shap_values = explainer.shap_values(X_test_subset)

#         # Store the SHAP values in a dictionary
#         shap_values_dict[model_name] = shap_values

#         # Save SHAP values to a file
#         shap_file = f"{model_name}_shap_values_pca.json"
#         # with open(shap_file, "w") as f:
#         #     json.dump(shap_values.tolist(), f)  # Convert numpy arrays to lists for JSON serialization

#         # Plot and save SHAP summary plot for each model
#         plt.figure()  # Create a new figure
#         shap.summary_plot(shap_values, X_test_subset, feature_names=X_test.columns)  # Prevent immediate display
    
#         plt.tight_layout()  # Adjust layout to fit everything
#         # plt.savefig(f"{model_name}_shap_summary_plot_from_json.png", format='png', dpi=300, bbox_inches='tight')  # Save the plot as a PNG file
#         plt.close()  # Close the plot to free up memory and avoid overlap
#             # Add the model to the processed list
#         processed_models.append(model_name)

#     except Exception as e:
#         print(f"An error occurred while processing {model_name}: {e}")

# print("SHAP analysis completed for all models.")


In [ ]:
import shap
import matplotlib.pyplot as plt

# Iterate through each fitted model to perform SHAP analysis
processed_models = []

# Summarize the background data using k-means with fewer clusters
background = shap.kmeans(X_test, 5)  # Reducing to 5 clusters for faster computation

# Use a subset of X_test for SHAP analysis
X_test_subset = X_test.sample(n=3, random_state=42)  

for model_name, model in fitted_models.items():
    try:
        if model_name != "SVM":  # Only process the SVM model
            continue
        print(f"Performing SHAP analysis for {model_name}...")

        # Use KernelExplainer for SVM
        explainer = shap.KernelExplainer(model.predict, background)
        
        # Calculate SHAP values using the subset of X_test
        shap_values = explainer.shap_values(X_test_subset)

        # Generate and save SHAP summary plot
        shap.summary_plot(shap_values, X_test_subset, feature_names=X_test.columns)
        plt.savefig(f"{model_name}_shap_summary_plot.png")  # Save the plot as a PNG file
        plt.close()  # Close the plot to avoid overlap in subsequent plots

        # Add the model to the processed list
        processed_models.append(model_name)

    except Exception as e:
        print(f"An error occurred while processing {model_name}: {e}")

print("SHAP analysis completed for all models.")


Performing SHAP analysis for SVM...


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
import json
import numpy as np
import shap
import pandas as pd
import matplotlib.pyplot as plt

# Function to load SHAP values from a JSON file
def load_shap_values(json_file):
    with open(json_file, "r") as f:
        shap_values_list = json.load(f)
    return shap.Explanation(values=np.array(shap_values_list))

# Example for a specific model
model_name = "SVM"  # Replace with the actual model name
shap_file = f"{model_name}_shap_values_boruta.json"

# Sample the test data
X_test_subset = X_test.sample(n=100, random_state=42)

try:
    shap_values = load_shap_values(shap_file)
    
    # Validate SHAP values
    if shap_values is None or shap_values.values.size == 0:
        raise ValueError("Loaded SHAP values are empty or not valid.")
    
    # Ensure SHAP values and feature data compatibility
    if X_test_subset.shape[1] != shap_values.values.shape[1]:
        raise ValueError("Mismatch between SHAP values and feature data.")
    
    # Generate and save SHAP summary plot
    plt.figure()
    shap.summary_plot(shap_values.values, X_test_subset, feature_names=X_test_subset.columns)
    
    # Save the plot as a PNG file
    plot_file = f"{model_name}_shap_summary_plot_from_json.png"
    plt.savefig(plot_file, format='png', dpi=300, bbox_inches='tight')
    plt.close()
    
    print(f"SHAP summary plot generated and saved as {plot_file}.")

except Exception as e:
    print(f"An error occurred: {e}")


In [ ]:
# import shap
# import json
# import numpy as np
# import matplotlib.pyplot as plt

# def load_shap_values_and_generate_plot(model_name, X_test):
#     """
#     Load SHAP values from a JSON file and generate a SHAP summary plot.

#     Parameters:
#     - model_name: str, name of the model whose SHAP values are to be loaded
#     - X_test: DataFrame, the feature matrix used for the SHAP analysis
#     """
#     # Load SHAP values from the JSON file
#     shap_file = f"{model_name}_shap_values.json"
#     try:
#         with open(shap_file, "r") as f:
#             shap_values_list = json.load(f)
#     except FileNotFoundError:
#         print(f"File {shap_file} not found for model {model_name}.")
#         return
#     except json.JSONDecodeError:
#         print(f"Error decoding JSON from {shap_file} for model {model_name}.")
#         return

#     # Convert the loaded SHAP values list back to a NumPy array
#     shap_values_array = np.array(shap_values_list)
 
#     # Create SHAP values object (needed for plotting)
#     shap_values = shap._explanation.Explanation(shap_values_array, base_values=None, data=X_test.values)

#     # Generate and save the SHAP summary plot
#     plt.figure()  # Create a new figure
#     shap.summary_plot(shap_values, X_test, feature_names=X_test.columns, show=False)  # Prevent immediate display

#     plt.tight_layout()  # Adjust layout to fit everything
#     plt.savefig(f"{model_name}_shap_summary_plot_from_json.png", format='png', dpi=300, bbox_inches='tight')  # Save the plot as a PNG file
#     plt.close()  # Close the plot to free up memory and avoid overlap

#     print(f"SHAP summary plot generated and saved for {model_name}.")

# # Example usage
# # Iterate through the processed models and generate plots from the saved JSON files
# for model_name in processed_models:
#     load_shap_values_and_generate_plot(model_name, X_test)


In [ ]:
# import matplotlib.pyplot as plt
# from sklearn.metrics import roc_curve, precision_recall_curve

# def plot_curves(models, X_train, X_test, Y_train, Y_test):
#     plt.rcParams.update({'font.size': 12})
    
#     # ROC Curve for Training Data
#     plt.figure(figsize=(7, 5))
#     for model_name, model in models.items():
#         if model_name == 'DNN':
#             train_pred_prob = model.predict(X_train).ravel()
#         else:
#             train_pred_prob = model.predict_proba(X_train)[:, 1]
        
#         fpr_train, tpr_train, _ = roc_curve(Y_train, train_pred_prob)
#         plt.plot(fpr_train, tpr_train, label=f'{model_name}')
    
#     plt.plot([0, 1], [0, 1], linestyle='--', color='black')  # Diagonal 45-degree line
#     plt.title('ROC Curve - Training Data')
#     plt.xlabel('False Positive Rate')
#     plt.ylabel('True Positive Rate')
#     plt.legend()
#     plt.savefig('roc_curve_train.png', dpi=300)
#     plt.show()

#     # ROC Curve for Testing Data
#     plt.figure(figsize=(7, 5))
#     for model_name, model in models.items():
#         if model_name == 'DNN':
#             test_pred_prob = model.predict(X_test).ravel()
#         else:
#             test_pred_prob = model.predict_proba(X_test)[:, 1]
        
#         fpr_test, tpr_test, _ = roc_curve(Y_test, test_pred_prob)
#         plt.plot(fpr_test, tpr_test, label=f'{model_name}')
    
#     plt.plot([0, 1], [0, 1], linestyle='--', color='black')  # Diagonal 45-degree line
#     plt.title('ROC Curve - Testing Data')
#     plt.xlabel('False Positive Rate')
#     plt.ylabel('True Positive Rate')
#     plt.legend()
#     plt.savefig('roc_curve_test.png', dpi=300)
#     plt.show()

#     # Precision-Recall Curve for Training Data
#     plt.figure(figsize=(7, 5))
#     for model_name, model in models.items():
#         if model_name == 'DNN':
#             train_pred_prob = model.predict(X_train).ravel()
#         else:
#             train_pred_prob = model.predict_proba(X_train)[:, 1]
        
#         precision_train, recall_train, _ = precision_recall_curve(Y_train, train_pred_prob)
#         plt.plot(recall_train, precision_train, label=f'{model_name}')
#     # plt.plot([1, 0], [0, 1], linestyle='--', color='black')
#     plt.title('Precision-Recall Curve - Training Data')
#     plt.xlabel('Recall')
#     plt.ylabel('Precision')
#     plt.legend()
#     plt.savefig('precision_recall_curve_train.png', dpi=300)
#     plt.show()

#     # Precision-Recall Curve for Testing Data
#     plt.figure(figsize=(7, 5))
#     for model_name, model in models.items():
#         if model_name == 'DNN':
#             test_pred_prob = model.predict(X_test).ravel()
#         else:
#             test_pred_prob = model.predict_proba(X_test)[:, 1]
        
#         precision_test, recall_test, _ = precision_recall_curve(Y_test, test_pred_prob)
#         plt.plot(recall_test, precision_test, label=f'{model_name}')
#     # plt.plot([1, 0], [0, 1], linestyle='--', color='black')
#     plt.title('Precision-Recall Curve - Testing Data')
#     plt.xlabel('Recall')
#     plt.ylabel('Precision')
#     plt.legend()
#     plt.savefig('precision_recall_curve_test.png', dpi=300)
#     plt.show()

#     print("All plots have been saved as separate PNG files with 300 dpi.")

In [ ]:
# print("Plotting ROC and Precision-Recall curves...")
# plot_curves(fitted_models, X_train, X_test, Y_train, Y_test)
# print("Plots saved successfully.")